#  Feed Forward Net 
100 Sample , batch size 20 -> no. of Iteration  = 100/20 =5 in one epoch
MNIST Data set
DataLoader, Transformation,
Multilayer Neural Net, Activiation Function
Loss and Optimizer
Training Loop (batch Traning)
Model Evaluation
GPU support

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transform
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
#device Config 
device  = torch.device('cuda' if not torch.cuda.is_available() else 'cpu')

# Later we will push our tensor to the device, it will make sure if we have gpu then we use Cuda else CPU

In [ ]:
#hyper Parameters
input_size = 784 # our images will be 28 * 28 on flating them we will get 784
hidden_size = 100 # random-> can be changed for better performance
num_classes = 10 # for each digit 
num_epochs = 10
batch_size = 100
learning_rate = 0.001


#Loading MNIST data 
train_dataset = torchvision.datasets.MNIST(root= './data', train = True, transform = transform.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root= './data', train = False, transform = transform.ToTensor()) 
#why we dnt have to download the test_Dataset??

train_loader = torch.utils.data.DataLoader(dataset= train_dataset, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset= test_dataset, batch_size = batch_size, shuffle = False)


In [ ]:
# Checking one batch
example = iter(train_loader)
samples, labels = example.next()
print(samples.shape, labels.shape)



```
100 samples
1 -> channel not 3 for RGB
28 * 28 is size of image 
```



In [ ]:
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.imshow(samples[i][0],cmap = 'gray')
  plt.show

In [ ]:
#Classifier - Fully Connnected NN with one hidden Layer

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 =nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)

    #Since for this multiClass Problem we are going to use Cross Entropy as loss so, We dnt have to use softmax function as the final layer.

    return

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [ ]:
#loss and Optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Training Loop
n_total_steps =  len(train_loader)

for epoch in range(num_epochs):

  for i, (images, labels) in enumerate(train_loader):
    #Reshape the images -> currently the shape of image is 100, 1, 28, 28 but we have set out input size as 784 so with 100 sample our images shape should be 100, 784
    #print(images.shape)
   # images = images.view(100, 784).to(device)
    images =  images.reshape(-1, 28 *28)
    # print(images.shape)
    labels = labels.to(device)


    #forward Pass
    y_pred = model(images)

    #loss
    loss = criterion(y_pred, labels)
   

    #backward 
    loss.backward()

    # updates the weights
    optimizer.step(
    optimizer.zero_grad()
    
    if (i+1) % 100 == 0:
      print(f'epoch = {epoch + 1}/{num_epochs}, step = {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

In [ ]:
#test
with torch.no_grad():
  n_correct  =0 
  n_sample = 0

  for images, labels in test_loader:
    images = images.view(100, 784).to(device)
    labels = labels.to(device)
    outputs = model(images) # model is now trained model
    #print(images)

    #value , index
    image,pred_label = torch.max(outputs,1)

    n_sample  += labels.shape[0]
    #print((pred_label == labels).sum().item())
    n_correct += (pred_label == labels).sum().item()
    
  acc= 100.0 * n_correct/n_sample
  print(f'accuracy = {acc} ')

